In [15]:
import altair as alt
from altair import datum
import pandas as pd
import os 

# Convert datasets into one

In [6]:
# Pull the data
co2 = pd.read_csv(os.getcwd() + '/Data/CO2.csv', sep=',')
temps = pd.read_csv(os.getcwd() + '/Data/GlobalLandTemperaturesByMajorCity.csv', sep=',')
hdi = pd.read_csv(os.getcwd() + '/Data/HDI.csv', sep=',')
relevant_columns = ['country', 'year', 'co2', 'AverageTemperature', 'population', 'share_global_co2',
                   'AverageTemperatureUncertainty', 'gdp', 'Latitude', 'Longitude']

# Modify the datasets to match, and eliminates non-June months
temps = temps.drop(temps[temps['dt'].str[6] != '6'].index)
temps = temps.rename(columns={'dt': 'year', 'Country': 'country'})
temps['year'] = temps['year'].apply(lambda x: int(x[0:4]))
hdi = hdi.rename(columns={'Year': 'year', 'Entity': 'country', 'Historical Index of Human Development (Prados de la Escosura)': 'HDI'})

# Unionise the DataFrames
data = pd.merge(co2, temps, on=['year', 'country'])
data = pd.merge(data, hdi, on=['year', 'country'])
data = data[relevant_columns]

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().